In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os

import scanpy as sc
import scirpy as ir
import anndata as ann
import numpy as np
import pandas as pd
import seaborn as sb
from tqdm import tqdm
import math
from scipy import stats, sparse

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as clrs

from matplotlib import rcParams
import matplotlib.gridspec as gridspec

In [ ]:
import sys
sys.path.append('..')
import utils.visualisation as utils_vis
import utils.representation as utils_rep

In [ ]:
ir.__version__

In [ ]:
path_figs = '../../figures/dextramer/'

sc.settings.set_figure_params(dpi=600)
sc._settings.ScanpyConfig(figdir=path_figs)
sc.settings.verbosity = 3
sc.set_figure_params(vector_friendly=True, color_map='viridis', transparent=True)
sb.set_style('whitegrid')
sc._settings.settings._vector_friendly=False

colormap = 'flare'
binding_mode = 'binding_ct'
dpi = 600

In [ ]:
adata = sc.read('../../data/dextramer/02_dex_annotated_cd8.h5ad')
adata.uns['log1p']['base'] = None

In [ ]:
path_figs = f'../../figures/dextramer/paper/'

# Data Settings

In [ ]:
epitopes = adata.uns['epitopes']
cite_ids = adata.uns['cite_ids']
custom_cite_ids = adata.uns['custom_cite_ids']
cite_ids_full = cite_ids.tolist() + custom_cite_ids.tolist()

In [ ]:
binding_mode = 'binding_ct'
time_order = ['P1', 'S1', 'S2', 'S3', 'T1', 'T2', 'T3', 'X3']
time_order_wo_X = ['P1', 'S1', 'S2', 'S3', 'T1', 'T2', 'T3']
cell = adata.uns['celltype']
DPI = 360

In [ ]:
specs = adata.obs['binding_ct'].astype(str).value_counts().index.tolist()
specs.remove('No binding')
specs_x = adata[adata.obs['time'].isin(['X3', 'extra'])].obs['binding_ct'].value_counts().index.tolist()
specs_x.remove('No binding')

In [ ]:
leiden_dpt_order = adata.obs.groupby('leiden_CD8')['dpt_pseudotime'].mean().sort_values().index.tolist()
leiden_dpt_order_wo11 = leiden_dpt_order.copy()
leiden_dpt_order_wo11.remove('11')

In [ ]:
leiden_order_tmp = leiden_dpt_order.copy()
for el in ['11', '12', '7', '9']:
    leiden_order_tmp.remove(el)
leiden_order_tmp

In [ ]:
epitope_2_donor = {
    'LTDEMIAQY': ['A04', 'A07', 'A08', 'A15', 'A16', 'HIM'],
    'QPYRVVVL': ['A02', 'A07', 'A08', 'A15', 'A19', 'HIM'],
    'YLQPRTFLL': ['A03', 'A07', 'A11', 'A25', 'A29'],
    'RLQSLQTYV': ['A03', 'A07', 'A11', 'A25', 'A29'],
    'VLNDILSRL': ['A03', 'A07', 'A11', 'A25', 'A29'],
    'KIADYNYKL': ['A03', 'A07', 'A11', 'A25', 'A29'],
    'YTNSFTRGVY': ['A04',  'A08', 'A15'],
    'NYNYLYRLF': ['A02', 'A05', 'A06', 'A15'],
    'ATDSLNNEY': ['A07', 'A15', 'HIM'],
    'CTELKLSDY': ['A07', 'A15', 'HIM'],
    'FLRGRAYGL': ['A07', 'A15', 'HIM'],
    'RAKFKQLL': ['A07', 'A15', 'HIM'],
    'IYKTPPIKDF': ['A02', 'A03', 'A04', 'A05', 'A06', 'A11', 'A29'],
    'SPRRARSVA': ['A02', 'A03', 'A04', 'A05', 'A06', 'A29', 'A11'],
    'FPQSAPHGV': ['A02', 'A03', 'A04', 'A05', 'A06', 'A29', 'A11'],
    'QYIKWPWYI': ['A02', 'A05', 'A06', 'A15'],
    'KCYGVSPTK': ['A03', 'A08', 'A29'], 
    'RLQSLQTYV': ['A03', 'A07', 'A11', 'A25', 'A29'],    
    'No binding': 'No binding'
}

# Colors

In [ ]:
sc.pl.umap(adata, color='leiden_CD8')
palette_leiden = dict(zip(adata.obs['leiden_CD8'].value_counts().index, 
                          adata.uns['leiden_CD8_colors']))
sc.pl.umap(adata, color='leiden_CD8', palette=palette_leiden)
palette_leiden_int = {int(k): v for k, v in palette_leiden.items()}

In [ ]:
sc.pl.umap(adata, color='binding_ct')
palette_epis = dict(zip(adata.obs['binding_ct'].cat.categories, 
                          adata.uns['binding_ct_colors']))
sc.pl.umap(adata, color='binding_ct', palette=palette_epis)
palette_epis_int = {i: v for i, v in enumerate(palette_epis.values())}

## UMAPs
### Clones

In [ ]:
clones = ['9130.0', '11251.0', '15599.0', '18755.0', '19327.0', '20205.0']

fig, ax = plt.subplots(1, 1, figsize=(5.75, 5))
sc.pl.umap(adata, ax=ax, show=False)
sc.pl.umap(adata[adata.obs['clone_id'].isin(clones)], 
           color='clone_id', s=len(adata)/1200*5,
           ax=ax, show=False)


plt.tight_layout()
plt.savefig(f'{path_figs}/umap_selected_clones.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/umap_selected_clones.png', bbox_inches='tight', dpi=DPI)
plt.show()

In [ ]:
def plot_clone_over_time(clone, donor, color=None):
    nrows = 1
    ncols = len(time_order_wo_X)
    fig, axes = plt.subplots(nrows, ncols, figsize=(5*ncols, 5*nrows))

    for i, t in enumerate(time_order_wo_X):
        ax = axes[i]
        sc.pl.umap(adata, ax=ax, show=False)

        adata_tmp = adata[(adata.obs['clone_id']==f'{clone}.0') 
                         & (adata.obs['time']==t) ].copy() 
        if donor is not None:
            adata_tmp = adata_tmp[adata_tmp.obs['donor']==donor].copy()
        if len(adata_tmp)>0:
            sc.pl.umap(adata_tmp, color='leiden_CD8', palette=['forestgreen']*20 if color is None else palette_leiden,
                       s=len(adata)/1200 *5,
                       ax=ax, show=False)
            ax.legend().remove()
        ax.set_xlabel(None)
        ax.set_ylabel(None)
        ax.set_title(t)
    axes[0].set_ylabel(f'{clone}.0' + (f' {donor}' if donor is not None else ''))
    plt.tight_layout()
    filename = f'umap_clone{clone}_{donor}_overtime{"_colorLeiden" if color is not None else ""}'
    plt.savefig(f'{path_figs}/{filename}.pdf', bbox_inches='tight', dpi=DPI)
    plt.savefig(f'{path_figs}/{filename}.png', bbox_inches='tight', dpi=DPI)
    plt.show()

In [ ]:
plot_clone_over_time('5251', None)

In [ ]:
plot_clone_over_time('7734', 'A08')
plot_clone_over_time('11095', 'A08')

In [ ]:
plot_clone_over_time('99', 'A04')
plot_clone_over_time('718', 'A04')
plot_clone_over_time('5918', 'A04')

In [ ]:
plot_clone_over_time('489', 'A15')
plot_clone_over_time('684', 'A15')
plot_clone_over_time('317', 'A15')

In [ ]:
plot_clone_over_time('5251', None, 'leiden')
plot_clone_over_time('99', None, 'leiden')
plot_clone_over_time('489', None, 'leiden')

### Which CiteSeq

In [ ]:
rcParams['figure.figsize'] = (5, 4.25)
adata.obs['has_cocktail'] = adata.obs['Hu.CD101'].notna()
adata.obs['has_costum'] = adata.obs['CD45RA'].notna()
adata.obs['Cite Markers'] = adata.obs.apply(lambda x: 'Cocktail' if x['has_cocktail'] 
                                          else 'Costum' if x['has_costum']
                                         else np.nan, axis=1)
sc.pl.umap(adata, color='Cite Markers', #palette={'False': 'lightgray', 'True': 'tab:orange'}, 
           show=False)
plt.tight_layout()
plt.savefig(f'{path_figs}/umap_whichCite.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/umap_whichCite.png', bbox_inches='tight', dpi=DPI)
plt.show()

### Genes

adata.obs['has_MKI67'] = (adata[:, 'MKI67'].X.A > 0).astype(str)
sc.tl.embedding_density(adata, groupby='has_MKI67')
plot = sc.pl.embedding_density(adata, key='umap_density_has_MKI67', group='True', color_map='coolwarm', show=False)
plot[0].set_title('MKI67 density')

plt.tight_layout()
plt.savefig(f'{path_figs}/umap_mki67_density.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/umap_mki67_density.png', bbox_inches='tight', dpi=DPI)
plt.show()

adata.obs['has_ifn_seumois'] = (adata.obs['ifn_seumois'].values > 0).astype(str)
sc.tl.embedding_density(adata, groupby='has_ifn_seumois')
plot = sc.pl.embedding_density(adata, key='umap_density_has_ifn_seumois', group='True', color_map='coolwarm', show=False)
plot[0].set_title('ifn_seumois density')

plt.tight_layout()
plt.savefig(f'{path_figs}/umap_ifn_seumois_density.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/umap_ifn_seumois_density.png', bbox_inches='tight', dpi=DPI)
plt.show()

In [ ]:
cmap = 'RdYlBu'

In [ ]:
rcParams['figure.figsize'] = (5.5, 5)

for gene in ['SELL', 'TCF7', 'GZMK', 'MKI67', 'CCL5', 'CD69', 'CXCR6']:
    plot = sc.pl.umap(adata, show=False)
    adata_tmp = adata[adata[:, gene].X.A>0]
    size = 5
    if len(adata_tmp) < 500:
        size = 15
    plot = sc.pl.umap(adata_tmp, color=gene, color_map=f'{cmap}_r', 
                      show=False, s=len(adata)/12000*size, ax=plot, #add_outline=True,
                      #vmin=adata[:, 'MKI67'].X.min()
                     )

    plt.tight_layout()
    plt.savefig(f'{path_figs}/umap_{gene}.pdf', bbox_inches='tight', dpi=DPI)
    plt.savefig(f'{path_figs}/umap_{gene}.png', bbox_inches='tight', dpi=DPI)
    plt.show()

In [ ]:
adata.obs['log10_clone_size'] = np.log10(adata.obs['clone_size'])

In [ ]:
rcParams['figure.figsize'] = (5.5, 5)

for gene in ['ifn_seumois', 'CD8 Cytotoxic_score', 'dpt_pseudotime', 'log10_clone_size']:
    plot = sc.pl.umap(adata, show=False)
    adata_tmp = adata[(adata.obs[gene].notna())]
    size = 5
    if len(adata_tmp) < 500:
        size = 15
    plot = sc.pl.umap(adata_tmp, color=gene, color_map=f'{cmap}_r', 
                      show=False, s=len(adata)/12000*size, ax=plot, #add_outline=True,
                      #vmin=adata[:, 'MKI67'].X.min()
                     )

    plt.tight_layout()
    plt.savefig(f'{path_figs}/umap_{gene}.pdf', bbox_inches='tight', dpi=DPI)
    plt.savefig(f'{path_figs}/umap_{gene}.png', bbox_inches='tight', dpi=DPI)
    plt.show()

In [ ]:
adata.X.min()

In [ ]:
plot = sc.pl.umap(adata, show=False)
adata_tmp = adata[(adata.obs['dpt_pseudotime'].notna())]
size = 5
if len(adata_tmp) < 500:
    size = 15
plot = sc.pl.umap(adata_tmp, color='dpt_pseudotime', color_map=f'{cmap}_r', 
                  show=False, s=len(adata)/12000*size, ax=plot,
                  vcenter=0.25
                 )

plt.tight_layout()
plt.savefig(f'{path_figs}/umap_dpt_pseudotime_centered.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/umap_dpt_pseudotime_centered.png', bbox_inches='tight', dpi=DPI)
plt.show()

In [ ]:
plot = sc.pl.umap(adata, show=False)
adata_tmp = adata[(adata.obs['dpt_pseudotime'].notna())]
size = 5
if len(adata_tmp) < 500:
    size = 15
plot = sc.pl.umap(adata_tmp, color='dpt_pseudotime', color_map=f'{cmap}_r', 
                  show=False, s=len(adata)/12000*size, ax=plot, 
                  vmax=0.6
                 )

plt.tight_layout()
plt.savefig(f'{path_figs}/umap_dpt_pseudotime_clipped.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/umap_dpt_pseudotime_clipped.png', bbox_inches='tight', dpi=DPI)
plt.show()

In [ ]:
adata.obs['log10_clone_size'] = np.log10(adata.obs['clone_size'])
adata.obs['ln_clone_size'] = np.log(adata.obs['clone_size'])
adata.obs[['log10_clone_size', 'ln_clone_size', 'clone_size', 'clone_size_donor']]

In [ ]:
for gene in ['ifn_seumois', 'CD8 Cytotoxic_score']:
    plot = sc.pl.umap(adata, show=False)
    adata_tmp = adata[(adata.obs[gene]>0)]
    size = 5
    if len(adata_tmp) < 500:
        size = 15
    plot = sc.pl.umap(adata_tmp, color=gene, color_map=f'{cmap}_r', 
                      show=False, s=len(adata)/12000*size, ax=plot, #add_outline=True,
                      #vmin=adata[:, 'MKI67'].X.min()
                     )

    plt.tight_layout()
    plt.savefig(f'{path_figs}/umap_{gene}_greater0.pdf', bbox_inches='tight', dpi=DPI)
    plt.savefig(f'{path_figs}/umap_{gene}_greater0.png', bbox_inches='tight', dpi=DPI)
    plt.show()

In [ ]:
rcParams['figure.figsize'] = (5.5, 5)

for gene in ['ifn_seumois']:
    plot = sc.pl.umap(adata, show=False)
    adata_tmp = adata[(adata.obs[gene].notna())
                     & (adata.obs['binding_ct'].isin(['LTDEMIAQY', 'YLQPRTFLL', 'KCYGVSPTK']))
                     & (adata.obs['donor']!='HIM')]
    size = 5
    if len(adata_tmp) < 500:
        size = 15
    plot = sc.pl.umap(adata_tmp, color=gene, color_map=f'{cmap}_r', 
                      show=False, s=len(adata)/12000*size, ax=plot, #add_outline=True,
                      vmin=adata[adata.obs[gene]>0].obs[gene].min(),
                      vmax=adata.obs[gene].max()
                     )

    plt.tight_layout()
    plt.savefig(f'{path_figs}/umap_{gene}_LtdYlqKcy_woHIM.pdf', bbox_inches='tight', dpi=DPI)
    plt.savefig(f'{path_figs}/umap_{gene}_LtdYlqKcy_woHIM.png', bbox_inches='tight', dpi=DPI)
    plt.show()

In [ ]:
def plot_correlation_tmp(df, x, y):
    name = 'All Clusters'
    if df['leiden_CD8'].nunique() == 1:
        name = f'Cluster {df["leiden_CD8"].unique()[0]}'
    plot = sb.scatterplot(data=df_tmp, x=x, y=y, 
                          s=1 if 'All' in name else 5,
                          hue='leiden_CD8' if 'All' in name else None,
                          palette=palette_leiden if 'All' in name else None)
    plot = sb.regplot(data=df_tmp, x=x, y=y, scatter=False, color='grey')
    
    r, p = stats.pearsonr(df_tmp[x].values, df_tmp[y].values)
    
    sb.despine(ax=plot)
    plot.grid(False)
    plot.set_title(f'{name} - r={r:.3f}, p-val={p:.3f}')
    if 'All' in name:
        plot.legend(bbox_to_anchor=(1.1, 0.4), loc='center left')
    
    plt.tight_layout()
    plt.savefig(f'{path_figs}/correlation_{x}_{y}_{name.replace(" ", "")}.pdf', bbox_inches='tight', dpi=DPI)
    plt.savefig(f'{path_figs}/correlation_{x}_{y}_{name.replace(" ", "")}.png', bbox_inches='tight', dpi=DPI)
    plt.show()

# Prepare data    
df_tmp = adata.obs[['leiden_CD8', 'ifn_seumois', 'dpt_pseudotime']].copy()
df_tmp['SELL'] = adata[:, 'SELL'].X.A

# All cells
plot_correlation_tmp(df_tmp, x='ifn_seumois', y='SELL')
plot_correlation_tmp(df_tmp, x='dpt_pseudotime', y='ifn_seumois')
plot_correlation_tmp(df_tmp, x='dpt_pseudotime', y='SELL')

# Only for cluster 8
df_tmp = df_tmp[df_tmp['leiden_CD8']=='8']    

plot_correlation_tmp(df_tmp, x='ifn_seumois', y='SELL')
plot_correlation_tmp(df_tmp, x='dpt_pseudotime', y='ifn_seumois')
plot_correlation_tmp(df_tmp, x='dpt_pseudotime', y='SELL')

### IFN for selected COV+ over time

In [ ]:
nrows = len(specs)
ncols = len(time_order_wo_X)


fig, axes = plt.subplots(nrows=1, ncols=ncols, figsize=(5*ncols, 5*1))
axes = axes.reshape((1, -1))
adata_s = adata[adata.obs['binding_ct'].isin(['LTDEMIAQY', 'YLQPRTFLL', 'KCYGVSPTK']) 
                & (adata.obs['donor']!='HIM')]

for j, t in enumerate(time_order_wo_X):
    adata_st = adata_s[adata_s.obs['time']==t]
    ax = axes[0][j]

    sc.pl.umap(adata, ax=ax, show=False)

    if len(adata_st) > 0:
        sc.pl.umap(adata_st, color='ifn_seumois', ax=ax, show=False, size=75, color_map=f'{cmap}_r',
                   vmin=adata.obs['ifn_seumois'].min(), vmax=adata.obs['ifn_seumois'].max())
    ax.set_xlabel(None)
    ax.set_ylabel(None)
    ax.set_title(None)
    
    if t != 'T3':
        ax.collections[1].colorbar.remove()

ax = axes[0][0]
ax.set_ylabel('ifn_seumois')

for j, t in enumerate(time_order_wo_X):
    ax = axes[0][j]
    ax.set_title(t)

plt.tight_layout()
plt.savefig(f'{path_figs}/umap_ifn_seumois_over_time_selected_woHIM.png', bbox_inches='tight', dpi=300,)
plt.savefig(f'{path_figs}/umap_ifn_seumois_over_time_selected_woHIM.pdf', bbox_inches='tight', dpi=300,)
plt.show()

### Dex+ per Epitope

In [ ]:
binding_epitopes = adata.obs['binding_ct'].value_counts().index.tolist()
binding_epitopes.remove('No binding')

fig, axes = plt.subplots(nrows=3, ncols=5, figsize=(5*5, 3*5))
axes = axes.reshape(-1)

for i, e in enumerate(binding_epitopes):
    ax = axes[i]
    sc.pl.umap(adata, ax=ax, show=False)
    adata_tmp = adata[adata.obs['binding_ct']==e] 
    sc.pl.umap(adata_tmp, color='binding_ct', ax=ax, show=False, size=75, palette=['black'])
    ax.legend().remove()
    ax.set_title(e)
    ax.set_ylabel(None)
    ax.set_xlabel(None)
    
axes[14].axis('off')
    
plt.suptitle(f'All timepoints')
plt.tight_layout()
plt.savefig(f'{path_figs}/umap_binding_separate_epitopes.pdf', bbox_inches='tight', dpi=300,)
plt.savefig(f'{path_figs}/umap_binding_separate_epitopes.png', bbox_inches='tight', dpi=300,)
plt.show()

In [ ]:
rcParams['figure.figsize'] = (5, 5)
binding_epitopes = adata.obs['binding_ct'].value_counts().index.tolist()
binding_epitopes.remove('No binding')

for i, e in enumerate(binding_epitopes):
    ax = sc.pl.umap(adata, show=False)
    adata_tmp = adata[adata.obs['binding_ct']==e] 
    sc.pl.umap(adata_tmp, color='binding_ct', ax=ax, show=False, size=75, palette=['black'])
    ax.legend().remove()
    ax.set_title(e)
    ax.set_ylabel(None)
    ax.set_xlabel(None)

    plt.tight_layout()
    plt.savefig(f'{path_figs}/umap_binding_{e}.pdf', bbox_inches='tight', dpi=300,)
    plt.savefig(f'{path_figs}/umap_binding_{e}.png', bbox_inches='tight', dpi=300,)
    plt.show()

In [ ]:
rcParams['figure.figsize'] = (5, 5)
binding_epitopes = adata.obs['binding_ct'].value_counts().index.tolist()
binding_epitopes.remove('No binding')

for i, e in enumerate(binding_epitopes):
    ax = sc.pl.umap(adata, show=False)
    adata_tmp = adata[adata.obs['binding_ct']==e] 
    sc.pl.umap(adata_tmp, color='leiden_CD8', ax=ax, show=False, size=75, palette=palette_leiden)
    ax.legend().remove()
    ax.set_title(e)
    ax.set_ylabel(None)
    ax.set_xlabel(None)

    plt.tight_layout()
    plt.savefig(f'{path_figs}/umap_binding_colored_{e}.pdf', bbox_inches='tight', dpi=300,)
    plt.savefig(f'{path_figs}/umap_binding_colored_{e}.png', bbox_inches='tight', dpi=300,)
    plt.show()

### Dex+ per Epitope over time

In [ ]:
nrows = len(specs)
ncols = len(time_order)


for i, s in enumerate(specs):
    fig, axes = plt.subplots(nrows=1, ncols=ncols, figsize=(5*ncols, 5*1))
    axes = axes.reshape((1, -1))
    adata_s = adata[adata.obs['binding_ct']==s]
    
    for j, t in enumerate(time_order):
        adata_st = adata_s[adata_s.obs['time']==t]
        ax = axes[0][j]
        
        sc.pl.umap(adata, ax=ax, show=False)
        
        if len(adata_st) > 0:
            sc.pl.umap(adata_st, color='binding_ct', ax=ax, show=False, size=75, palette=['black'])
        ax.legend().remove()
        ax.set_xlabel(None)
        ax.set_ylabel(None)
        ax.set_title(None)
    
    ax = axes[0][0]
    ax.set_ylabel(s)

    for j, t in enumerate(time_order):
        ax = axes[0][j]
        ax.set_title(t)

    plt.tight_layout()
    plt.savefig(f'{path_figs}/umap_binding_over_time_{s}.png', bbox_inches='tight', dpi=300,)
    plt.savefig(f'{path_figs}/umap_binding_over_time_{s}.pdf', bbox_inches='tight', dpi=300,)
    plt.show()

In [ ]:
nrows = len(specs)
ncols = len(time_order)


for i, s in enumerate(specs):
    fig, axes = plt.subplots(nrows=1, ncols=ncols, figsize=(5*ncols, 5*1))
    axes = axes.reshape((1, -1))
    adata_s = adata[adata.obs['binding_ct']==s]
    
    for j, t in enumerate(time_order):
        adata_st = adata_s[adata_s.obs['time']==t]
        ax = axes[0][j]
        
        sc.pl.umap(adata, ax=ax, show=False)
        
        if len(adata_st) > 0:
            sc.pl.umap(adata_st, color='leiden_CD8', ax=ax, show=False, size=75, palette=palette_leiden)
        ax.legend().remove()
        ax.set_xlabel(None)
        ax.set_ylabel(None)
        ax.set_title(None)
    
    ax = axes[0][0]
    ax.set_ylabel(s)

    for j, t in enumerate(time_order):
        ax = axes[0][j]
        ax.set_title(t)

    plt.tight_layout()
    plt.savefig(f'{path_figs}/umap_binding_over_time_colered_{s}.png', bbox_inches='tight', dpi=300,)
    plt.savefig(f'{path_figs}/umap_binding_over_time_colered_{s}.pdf', bbox_inches='tight', dpi=300,)
    plt.show()

In [ ]:
nrows = len(specs)
ncols = len(time_order_wo_X)


for i, d in enumerate(['A04', 'A08', 'A15']):
    fig, axes = plt.subplots(nrows=1, ncols=ncols, figsize=(5*ncols, 5*1))
    axes = axes.reshape((1, -1))
    adata_s = adata[(adata.obs['binding_ct']=='LTDEMIAQY')
                   & (adata.obs['donor']==d)]
    
    for j, t in enumerate(time_order_wo_X):
        adata_st = adata_s[adata_s.obs['time']==t]
        ax = axes[0][j]
        
        sc.pl.umap(adata, ax=ax, show=False)
        
        if len(adata_st) > 0:
            sc.pl.umap(adata_st, color='leiden_CD8', ax=ax, show=False, size=75, palette=palette_leiden)
        ax.legend().remove()
        ax.set_xlabel(None)
        ax.set_ylabel(None)
        ax.set_title(None)
    
    ax = axes[0][0]
    ax.set_ylabel(d)

    for j, t in enumerate(time_order_wo_X):
        ax = axes[0][j]
        ax.set_title(t)

    plt.tight_layout()
    plt.savefig(f'{path_figs}/umap_binding_over_time_colered_LTDEMIAQY_{d}.png', bbox_inches='tight', dpi=300,)
    plt.savefig(f'{path_figs}/umap_binding_over_time_colered_LTDEMIAQY_{d}.pdf', bbox_inches='tight', dpi=300,)
    plt.show()

In [ ]:
nrows = len(specs)
ncols = len(time_order_wo_X)


for i, d in enumerate(['A11', 'A29']):
    fig, axes = plt.subplots(nrows=1, ncols=ncols, figsize=(5*ncols, 5*1))
    axes = axes.reshape((1, -1))
    adata_s = adata[(adata.obs['binding_ct']=='YLQPRTFLL')
                   & (adata.obs['donor']==d)]
    
    for j, t in enumerate(time_order_wo_X):
        adata_st = adata_s[adata_s.obs['time']==t]
        ax = axes[0][j]
        
        sc.pl.umap(adata, ax=ax, show=False)
        
        if len(adata_st) > 0:
            sc.pl.umap(adata_st, color='leiden_CD8', ax=ax, show=False, size=75, palette=palette_leiden)
        ax.legend().remove()
        ax.set_xlabel(None)
        ax.set_ylabel(None)
        ax.set_title(None)
    
    ax = axes[0][0]
    ax.set_ylabel(d)

    for j, t in enumerate(time_order_wo_X):
        ax = axes[0][j]
        ax.set_title(t)

    plt.tight_layout()
    plt.savefig(f'{path_figs}/umap_binding_over_time_colered_YLQPRTFLL_{d}.png', bbox_inches='tight', dpi=300,)
    plt.savefig(f'{path_figs}/umap_binding_over_time_colered_YLQPRTFLL_{d}.pdf', bbox_inches='tight', dpi=300,)
    plt.show()

In [ ]:
nrows = len(specs)
ncols = len(time_order_wo_X)


for i, d in enumerate(['A08']):
    fig, axes = plt.subplots(nrows=1, ncols=ncols, figsize=(5*ncols, 5*1))
    axes = axes.reshape((1, -1))
    adata_s = adata[(adata.obs['binding_ct']=='KCYGVSPTK')
                   & (adata.obs['donor']==d)]
    
    for j, t in enumerate(time_order_wo_X):
        adata_st = adata_s[adata_s.obs['time']==t]
        ax = axes[0][j]
        
        sc.pl.umap(adata, ax=ax, show=False)
        
        if len(adata_st) > 0:
            sc.pl.umap(adata_st, color='leiden_CD8', ax=ax, show=False, size=75, palette=palette_leiden)
        ax.legend().remove()
        ax.set_xlabel(None)
        ax.set_ylabel(None)
        ax.set_title(None)
    
    ax = axes[0][0]
    ax.set_ylabel(d)

    for j, t in enumerate(time_order_wo_X):
        ax = axes[0][j]
        ax.set_title(t)

    plt.tight_layout()
    plt.savefig(f'{path_figs}/umap_binding_over_time_colered_KCYGVSPTK_{d}.png', bbox_inches='tight', dpi=300,)
    plt.savefig(f'{path_figs}/umap_binding_over_time_colered_KCYGVSPTK_{d}.pdf', bbox_inches='tight', dpi=300,)
    plt.show()

## Violin plots
### LTD+YLQ+KCY wo HIM

In [ ]:
rcParams['figure.figsize'] = (6, 4)
scores = ['ifn_seumois', 'IFNG', 'MKI67']

for s in scores:
    adata_tmp = adata[(adata.obs['donor']!='HIM')
                  & (adata.obs['binding_ct'].isin(['LTDEMIAQY', 'YLQPRTFLL', 'KCYGVSPTK']))]
    plot = sc.pl.violin(adata_tmp, groupby='time', keys=s, show=False)
    sb.despine(ax=plot)
    plot.grid(False)
    plt.tight_layout()
    plt.savefig(f'{path_figs}/violin_time_{s}.pdf', bbox_inches='tight', dpi=DPI)
    plt.savefig(f'{path_figs}/violin_time_{s}.png', bbox_inches='tight', dpi=DPI)
    plt.show()

### MKI67 + IFNG by leiden

In [ ]:
rcParams['figure.figsize'] = (6, 4)

for n, ad in [('All Donors', adata), ('Without HIM', adata[adata.obs['donor']!='HIM'])]:
    df_tmp = ad.obs[['leiden_CD8', 'donor']].copy()
    df_tmp['MKI67'] = ad[:, 'MKI67'].X.A
    df_tmp['IFNG'] = ad[:, 'IFNG'].X.A

    df_tmp = pd.melt(df_tmp, id_vars=['leiden_CD8', 'donor'], var_name='Gene', value_name='value')

    sb.violinplot(data=df_tmp, x='leiden_CD8', y='value', hue='Gene', order=leiden_dpt_order,
                  inner=None,
                  split=True, palette='pastel', scale='width')
    plot = sb.stripplot(data=df_tmp, x='leiden_CD8', y='value', hue='Gene', palette='dark',
                         order=leiden_dpt_order, dodge=True, size=1.5)
    sb.despine(ax=plot)
    plot.grid(False)
    plot.set_title(n)

    handles, labels = plot.get_legend_handles_labels()
    new_handles = handles[:-2]
    new_labels = labels[:-2]
    plot.legend(handles=new_handles, labels=new_labels)
    
    plt.tight_layout()
    plt.savefig(f'{path_figs}/violin_mki67ifgn_{n.replace(" ", "")}.pdf', bbox_inches='tight', dpi=DPI)
    plt.savefig(f'{path_figs}/violin_mki67ifgn_{n.replace(" ", "")}.png', bbox_inches='tight', dpi=DPI)
    plt.show()

## Pseudo Time

In [ ]:
rcParams['figure.figsize'] = (6, 4)

df_tmp = adata.obs[['leiden_CD8', 'dpt_pseudotime']]
df_tmp = df_tmp[df_tmp['leiden_CD8']!='11']
sb.violinplot(data=df_tmp, x='leiden_CD8', y='dpt_pseudotime', scale='width', inner=None, order=leiden_order_tmp,
             palette=palette_leiden)
plot = sb.stripplot(data=df_tmp, x='leiden_CD8', y='dpt_pseudotime', dodge=True, order=leiden_order_tmp,
             jitter=0.25, size=0.5, color='black', alpha=0.25)
sb.despine(ax=plot)
plot.grid(False)

plt.tight_layout()
plt.savefig(f'{path_figs}/violin_leiden_dpt.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/violin_leiden_dpt.png', bbox_inches='tight', dpi=DPI)
plt.show()

In [ ]:
df_tmp = adata.obs[['leiden_CD8', 'dpt_pseudotime']]
sb.violinplot(data=df_tmp, x='leiden_CD8', y='dpt_pseudotime', scale='width', inner=None, order=leiden_dpt_order,
             palette=palette_leiden)
plot = sb.stripplot(data=df_tmp, x='leiden_CD8', y='dpt_pseudotime', dodge=True, order=leiden_dpt_order,
             jitter=0.25, size=0.5, color='black', alpha=0.25)
sb.despine(ax=plot)
plot.grid(False)

plt.tight_layout()
plt.savefig(f'{path_figs}/violin_leiden_dpt_allClusters.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/violin_leiden_dpt_allClusters.png', bbox_inches='tight', dpi=DPI)
plt.show()

## Scatterplots
### Pseudotime vs genes

In [ ]:
leiden_order_tmp

In [ ]:
rcParams['figure.figsize'] = (5, 5)

def plot_dpt_gene(gene):
    adata_tmp = adata[adata.obs['leiden_CD8'].isin(leiden_order_tmp)]
    plot = sc.pl.scatter(adata_tmp, x='dpt_pseudotime', y=gene, color='leiden_CD8',
                         size=20, show=False)
    sb.despine(ax=plot)
    plot.grid(False)
    plot.set_title(None)
    
for g in ['MKI67', 'SELL', 'TCF7', 'IFN Response_score', 'CD8 Cytotoxic_score', 'ifn_seumois']:
    plot_dpt_gene(g)
    plt.tight_layout()
    plt.savefig(f'{path_figs}/scatter_dpt_{g}.pdf', bbox_inches='tight', dpi=DPI)
    plt.savefig(f'{path_figs}/scatter_dpt_{g}.png', bbox_inches='tight', dpi=DPI)
    plt.show()

In [ ]:
rcParams['figure.figsize'] = (6, 4)

df_tmp = adata.obs[['leiden_CD8', 'ifn_seumois']].copy()
shift = 1.25
df_tmp['ifn_seumois'] = df_tmp['ifn_seumois'] + shift

df_tmp['MKI67'] = adata[:, 'MKI67'].X.A

df_tmp = df_tmp.melt(id_vars=['leiden_CD8'], value_vars=['ifn_seumois', 'MKI67'], var_name='score', value_name='value')

fig = plt.figure()
ax = fig.add_subplot(111)
ax2 = ax.twinx()

sb.barplot(data=df_tmp[df_tmp['score']=='ifn_seumois'], x='leiden_CD8', y='value', hue='score', ax=ax2, 
           hue_order=[ 'MKI67', 'ifn_seumois'], palette='pastel', order=leiden_order_tmp)
sb.barplot(data=df_tmp[df_tmp['score']=='MKI67'], x='leiden_CD8', y='value', hue='score', ax=ax, 
           hue_order=[ 'MKI67', 'ifn_seumois'], palette='pastel', order=leiden_order_tmp)

ax.grid(False)
ax2.grid(False)
sb.despine(ax=ax, right=False)
sb.despine(ax=ax2, right=False)

ax2.legend().remove()
ax.legend()

ax2.set_ylabel('ifn_seumois')
ax2.set_yticklabels([el-shift for el in ax2.get_yticks()])
ax.set_ylabel('MKI67')

plt.tight_layout()
plt.savefig(f'{path_figs}/bar_mki67ifgSeumoi.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/bar_mki67ifgSeumois.png', bbox_inches='tight', dpi=DPI)
plt.show()

In [ ]:
rcParams['figure.figsize'] = (6, 4)

df_tmp = adata.obs[['leiden_CD8']].copy()

df_tmp['SELL'] = adata[:, 'SELL'].X.A
df_tmp['TCF7'] = adata[:, 'TCF7'].X.A

df_tmp = df_tmp.melt(id_vars=['leiden_CD8'], value_vars=['TCF7', 'SELL'], var_name='score', value_name='value')

fig = plt.figure()
ax = fig.add_subplot(111)
ax2 = ax.twinx()

sb.barplot(data=df_tmp[df_tmp['score']=='TCF7'], x='leiden_CD8', y='value', hue='score', ax=ax2, 
           hue_order=[ 'SELL', 'TCF7'], palette='pastel', order=leiden_order_tmp)
sb.barplot(data=df_tmp[df_tmp['score']=='SELL'], x='leiden_CD8', y='value', hue='score', ax=ax, 
           hue_order=[ 'SELL', 'TCF7'], palette='pastel', order=leiden_order_tmp)

ax.grid(False)
ax2.grid(False)
sb.despine(ax=ax, right=False)
sb.despine(ax=ax2, right=False)

ax2.legend().remove()
ax.legend()

ax2.set_ylabel('TCF7')
ax.set_ylabel('SELL')

plt.tight_layout()
plt.savefig(f'{path_figs}/bar_SellTcf7.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/bar_SellTcf7.png', bbox_inches='tight', dpi=DPI)
plt.show()

In [ ]:
rcParams['figure.figsize'] = (6, 4)

df_tmp = adata.obs[['leiden_CD8']].copy()

df_tmp['GZMK'] = adata[:, 'GZMK'].X.A

df_tmp = df_tmp.melt(id_vars=['leiden_CD8'], value_vars=['GZMK'], var_name='score', value_name='value')

fig = plt.figure()
ax = fig.add_subplot(111)

sb.barplot(data=df_tmp[df_tmp['score']=='GZMK'], x='leiden_CD8', y='value', hue='score', ax=ax, 
           palette='pastel', order=leiden_order_tmp)

ax.grid(False)
sb.despine(ax=ax, right=False)

ax.legend().remove()
ax.set_ylabel('GZMK')

plt.tight_layout()
plt.savefig(f'{path_figs}/bar_GZMK.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/bar_GZMK.png', bbox_inches='tight', dpi=DPI)
plt.show()

In [ ]:
rcParams['figure.figsize'] = (6, 4)

df_tmp = adata.obs[['leiden_CD8', 'CD8 Cytotoxic_score']].copy()
shift = 0.4
df_tmp['CD8 Cytotoxic_score'] = df_tmp['CD8 Cytotoxic_score'] + shift

df_tmp['CCL5'] = adata[:, 'CCL5'].X.A

df_tmp = df_tmp.melt(id_vars=['leiden_CD8'], value_vars=['CD8 Cytotoxic_score', 'CCL5'], var_name='score', value_name='value')

fig = plt.figure()
ax = fig.add_subplot(111)
ax2 = ax.twinx()

sb.barplot(data=df_tmp[df_tmp['score']=='CD8 Cytotoxic_score'], x='leiden_CD8', y='value', hue='score', ax=ax2, 
           hue_order=[ 'CCL5', 'CD8 Cytotoxic_score'], palette='pastel', order=leiden_order_tmp)
sb.barplot(data=df_tmp[df_tmp['score']=='CCL5'], x='leiden_CD8', y='value', hue='score', ax=ax, 
           hue_order=[ 'CCL5', 'CD8 Cytotoxic_score'], palette='pastel', order=leiden_order_tmp)

ax.grid(False)
ax2.grid(False)
sb.despine(ax=ax, right=False)
sb.despine(ax=ax2, right=False)

ax2.legend().remove()
ax.legend(bbox_to_anchor=(0, 1.1), loc='upper left')

ax2.set_ylabel('CD8 Cytotoxic_score')
ax2.set_yticklabels([f'{el-shift:.1f}' for el in ax2.get_yticks()])
ax.set_ylabel('CCL5')

plt.tight_layout()
plt.savefig(f'{path_figs}/bar_Ccl5Cytotoxic.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/bar_Ccl5Cytotoxic.png', bbox_inches='tight', dpi=DPI)
plt.show()

## Stacked Barplot
### Leiden over time

In [ ]:
def plot_frac_leiden(adata, epitope, times, ax):
    df_count = adata[adata.obs['binding_ct']==epitope].obs
    df_count = df_count.groupby('time')['leiden_CD8'].value_counts(normalize=True).unstack()
    for t in times:
        if t not in df_count.index:
            df_count.loc[t] = 0
    df_count = df_count.loc[times]
    df_count.columns = df_count.columns.astype(str)
    for l in adata.obs['leiden_CD8'].unique():
        if l not in df_count.columns:
            df_count[l] = 0
    #df_count = df_count.sort_index()
    df_count.columns = df_count.columns.astype(int)
    df_count = df_count[sorted(df_count.columns)]
    plot = df_count.plot(kind='bar', stacked=True, ax=ax, ylim=[0, 1], color=palette_leiden_int)
    sb.despine(ax=plot)
    plot.set_title(epitope)
    plot.grid(False)
    plot.set_xlabel(None)
    plot.set_xticklabels(plot.get_xticklabels(), rotation=0)
    plot.legend().remove()
    

nrows = 3 
ncols = 5

fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*5, nrows*5))
axes = axes.reshape(-1)

for i, e in enumerate(specs):
    plot_frac_leiden(adata[adata.obs['donor']!='HIM'], e, time_order_wo_X, axes[i])
    
plt.tight_layout()
plt.savefig(f'{path_figs}/leiden_over_time_per_specificity.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/leiden_over_time_per_specificity.png', bbox_inches='tight', dpi=DPI)
plt.show()

In [ ]:
nrows = 1 
ncols = 4

fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*5, nrows*5))
axes = axes.reshape(-1)

for i, e in enumerate(specs_x):
    plot_frac_leiden(adata, e, ['X3', 'extra'], axes[i])
    
plt.tight_layout()
plt.savefig(f'{path_figs}/leiden_over_time_per_specificity_extraTimes.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/leiden_over_time_per_specificity_extraTimes.png', bbox_inches='tight', dpi=DPI)
plt.show()

### PseudoTime

In [ ]:
n_bins = 10
bins = [1/n_bins * i for i in range(n_bins)]
adata.obs['pseudotime_bins'] = np.digitize(adata.obs['dpt_pseudotime'].values, bins)
adata.obs['pseudotime_bins'] = adata.obs['pseudotime_bins'].apply(lambda x: f'-{x/10}')
bins = sorted(adata.obs['pseudotime_bins'].unique())

nrows = len(specs)
ncols = len(time_order_wo_X)

fig, axes = plt.subplots(nrows, ncols, figsize=(5*ncols, 5*nrows))

for i, e in enumerate(specs):
    for j, t in enumerate(time_order_wo_X):
        ax = axes[i][j]
        df_counts = adata[(adata.obs['binding_ct']==e)
                         & (adata.obs['time']==t)].obs[['pseudotime_bins', 'leiden_CD8']]
        df_counts = df_counts[df_counts['leiden_CD8'].isin(leiden_order_tmp)]
        
        if len(df_counts) == 0:
            sb.despine(ax=ax, left=True, bottom=True)
            ax.grid(False)
            ax.set_xticklabels([])
            ax.set_yticklabels([])
            continue
        df_counts = df_counts.groupby('pseudotime_bins')['leiden_CD8'].value_counts()
        df_counts = df_counts.unstack().fillna(0)
        for b in bins:
            if b not in df_counts.index:
                df_counts.loc[b] = 0
        df_counts = df_counts.loc[bins]

        plot = df_counts.plot(kind='bar', stacked=True, color=palette_leiden, ax=ax)
        sb.despine(ax=plot)
        plot.grid(False)
        plot.legend().remove()
        plot.set_xlabel(None)
    
for i, e in enumerate(specs):
    axes[i][0].set_ylabel(e)
    
for j, t in enumerate(time_order_wo_X):
    axes[0][j].set_title(t)
    
plt.tight_layout()
plt.savefig(f'{path_figs}/amountCells_pseudotime_by_epitope.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/amountCells_pseudotime_by_epitope.png', bbox_inches='tight', dpi=DPI)
plt.show()

## Gini Coefficient

In [ ]:
import skbio
def gini(*args, **kwargs):
    # Needed to write a wrapper passable to scirpy, since skbio standardly uses rectangular estimation
    # This leads to negative Ginis for cases where all cts are distributed equally.
    return skbio.diversity.alpha.gini_index(*args, **kwargs, method='trapezoids')

In [ ]:
def calculate_gini(adata_in, condition):
    adata_tmp = adata_in[adata_in.obs['clone_id'].notna() 
                 & (adata_in.obs['clone_id']!='nan')].copy()
    adata_tmp.obs['clone_id'] = adata_tmp.obs['clone_id'].astype(str)
    diversity = ir.tl.alpha_diversity(adata_tmp, groupby=condition, target_col='clone_id', 
                                      metric=gini, inplace=False)
    diversity = diversity.rename(columns={0: 'Gini'})
    diversity[condition] = diversity.index
    return diversity

### Over leiden

In [ ]:
rcParams['figure.figsize'] = (5, 5)
gini_leiden = calculate_gini(adata, 'leiden_CD8')
plot = sb.barplot(data=gini_leiden, x='leiden_CD8', y='Gini', palette=palette_leiden, 
                  order=leiden_dpt_order)
sb.despine(ax=plot)
plot.grid(False)
plot.set_title('Diversity - All cells')
plt.tight_layout()
plt.savefig(f'{path_figs}/gini_leiden_allCells.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/gini_leiden_allCells.png', bbox_inches='tight', dpi=DPI)
plt.show()
gini_leiden

In [ ]:
gini_leiden = calculate_gini(adata[adata.obs['binding_ct']!='No binding'], 'leiden_CD8')
plot = sb.barplot(data=gini_leiden, x='leiden_CD8', y='Gini', palette=palette_leiden, 
                  order=leiden_dpt_order)
sb.despine(ax=plot)
plot.grid(False)
plot.set_title('Diversity - Dex+')
plt.tight_layout()
plt.savefig(f'{path_figs}/gini_leiden_dex+.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/gini_leiden_dex+.png', bbox_inches='tight', dpi=DPI)
plt.show()

### Longitudinal

In [ ]:
adata[adata.obs['donor'].isin(epitope_2_donor['LTDEMIAQY'])].obs.groupby(['donor', 'time'])['clone_id'].nunique()

In [ ]:
ginis_time = []
for donor in epitope_2_donor['LTDEMIAQY']:
    if donor in ['HIM', 'A07']:
        continue
    adata_time = adata[(adata.obs['binding_ct']=='LTDEMIAQY')
                      & (adata.obs['donor']==donor)]
    gini_time = calculate_gini(adata_time, 'time').reset_index(drop=True)
    gini_time['Donor'] = donor
    ginis_time.append(gini_time)
ginis_time = pd.concat(ginis_time).reset_index(drop=True)

print(len(ginis_time))
for _, row in ginis_time.iterrows():
    d = row['Donor']
    t = row['time']
    df_tmp = adata[(adata.obs['binding_ct']=='LTDEMIAQY')
                      & (adata.obs['donor']==d)
                  & (adata.obs['time']==t)].obs
    if df_tmp['clone_id'].nunique()<2:
        ginis_time = ginis_time[(ginis_time['Donor']!=d) | (ginis_time['time']!=t)].copy()
print(len(ginis_time))
        
ginis_time['time'] = pd.Categorical(ginis_time['time'], categories=time_order_wo_X, ordered=True)
ginis_time = ginis_time.sort_values(by='time')
plot = sb.barplot(data=ginis_time, x='time', y='Gini', 
                  palette=['white'], edgecolor='black',
                  order=time_order_wo_X)
sb.lineplot(data=ginis_time, x='time', y='Gini', hue='Donor', sort=False, zorder=10)
sb.stripplot(data=ginis_time, x='time', y='Gini', hue='Donor', order=time_order_wo_X, jitter=False, ax=plot)

handles, labels = plot.get_legend_handles_labels()
plot.legend(handles[4:], labels[4:], loc='upper left', bbox_to_anchor=(1, 1))

sb.despine(ax=plot)
plot.grid(False)
plot.set_title('LTD wo HIM')

plt.tight_layout()
plt.savefig(f'{path_figs}/gini_time_ltd.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/gini_time_ltd.png', bbox_inches='tight', dpi=DPI)
plt.show()

In [ ]:
rcParams['figure.figsize'] = (5, 5)
ginis_time = []

for donor in adata.obs['donor'].unique():
    if donor == 'HIM':
        continue
    adata_time = adata[(~adata.obs['binding_ct'].isin(['No binding', 'CTELKLSDY', 'FLRGRAYGL', 'RAKFKQLL']))
                      & (adata.obs['donor']==donor)]
    gini_time = calculate_gini(adata_time, 'time').reset_index(drop=True)
    gini_time['Donor'] = donor
    ginis_time.append(gini_time)
ginis_time = pd.concat(ginis_time).reset_index(drop=True)

print(len(ginis_time))
for _, row in ginis_time.iterrows():
    d = row['Donor']
    t = row['time']
    df_tmp = adata[(~adata.obs['binding_ct'].isin(['No binding', 'CTELKLSDY', 'FLRGRAYGL', 'RAKFKQLL']))
                      & (adata.obs['donor']==d)
                  & (adata.obs['time']==t)].obs
    if df_tmp['clone_id'].nunique()<2:
        ginis_time = ginis_time[(ginis_time['Donor']!=d) | (ginis_time['time']!=t)].copy()
print(len(ginis_time))

ginis_time['time'] = pd.Categorical(ginis_time['time'], categories=time_order_wo_X, ordered=True)
ginis_time = ginis_time.sort_values(by='time')
plot = sb.barplot(data=ginis_time, x='time', y='Gini', 
                  palette=['white'], edgecolor='black',
                  order=time_order_wo_X)
#sb.lineplot(data=ginis_time, x='time', y='Gini', hue='Donor', sort=False, zorder=10)
sb.stripplot(data=ginis_time, x='time', y='Gini', hue='Donor', order=time_order_wo_X, jitter=False, ax=plot)
sb.despine(ax=plot)
plot.grid(False)
plot.set_title('Cov+ wo HIM')

handles, labels = plot.get_legend_handles_labels()
plot.legend(handles, labels, bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.savefig(f'{path_figs}/gini_time_cov+.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/gini_time_cov+.png', bbox_inches='tight', dpi=DPI)
plt.show()

### Acute vs Memory

In [ ]:
rcParams['figure.figsize'] = (5, 5)
adata_state = adata[(adata.obs['binding_ct']=='LTDEMIAQY')
                   & (adata.obs['time']!='P1')
                   & (adata.obs['donor']!='HIM')].copy()
adata_state.obs['time_state'] = adata_state.obs['time'].apply(lambda x: 'A' if x in ['S1', 'T1'] else 'M')
# only keep donor with at clones in both
adata_state = adata_state[~adata_state.obs['donor'].isin(['A07', 'A16'])].copy()

ginis_state = []
for donor in epitope_2_donor['LTDEMIAQY']:

    adata_sd = adata_state[adata_state.obs['donor']==donor]
    gini_state = calculate_gini(adata_sd, 'time_state').reset_index(drop=True)
    gini_state['time_state'] = gini_state['time_state'] + '_LTD'
    gini_state['Donor'] = donor
    ginis_state.append(gini_state)

adata_state = adata[(adata.obs['binding_ct']=='CTELKLSDY')].copy()
for donor in epitope_2_donor['CTELKLSDY']:
    adata_sd = adata_state[adata_state.obs['donor']==donor]
    gini_state = calculate_gini(adata_sd, 'binding_ct').reset_index(drop=True)
    gini_state['time_state'] = gini_state['binding_ct']
    gini_state['Donor'] = donor
    ginis_state.append(gini_state)
    
ginis_state = pd.concat(ginis_state).reset_index(drop=True)

sb.barplot(data=ginis_state, x='time_state', y='Gini', palette=['white'], edgecolor='black')
sb.lineplot(data=ginis_state[ginis_state['binding_ct'].isna()], x='time_state', y='Gini', hue='Donor', zorder=10)
plot = sb.stripplot(data=ginis_state, x='time_state', y='Gini', hue='Donor', 
                    jitter=False)
sb.despine(ax=plot)
plot.grid(False)

plot.set_xticklabels(plot.get_xticklabels(), rotation=90)
plot.set_xlabel(None)

handles, labels = plot.get_legend_handles_labels()
plot.legend(handles[4:], labels[4:], loc='upper left', bbox_to_anchor=(1, 1))

plt.tight_layout()
plt.savefig(f'{path_figs}/gini_acuteMemory.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/gini_acuteMemory.png', bbox_inches='tight', dpi=DPI)
plt.show()

In [ ]:
df_acc = ginis_state[ginis_state['time_state']=='A_LTD'].sort_values('Donor')
df_mem = ginis_state[ginis_state['time_state']=='M_LTD'].sort_values('Donor')
tresult = stats.ttest_rel(df_acc['Gini'].values, df_mem['Gini'].values)
print(tresult)

stats.shapiro(df_acc['Gini'].values-df_mem['Gini'].values)

In [ ]:
adata.obs['donor'].nunique()

In [ ]:
adata.obs['binding_ct'].value_counts()

### Expansion

In [ ]:
rcParams['figure.figsize'] = (7, 4)

cmap_donor_a07 = {
    'Other': 'firebrick',
    'A07': 'black'
}
def plot_size_amount_clones(adata_tmp, title):
    df_tmp = adata_tmp.obs[['donor', 'clone_id', 'clone_size_donor']].copy()
    print(len(df_tmp))
    df_tmp = df_tmp.drop_duplicates()
    df_tmp = df_tmp.groupby('donor')['clone_size_donor'].value_counts().unstack().fillna(0)
    steps = [0, 1, 2, 3, 5, 10, 15, 20, 999999]
    dfs = []
    for i, s in enumerate(steps[:-1]):
        df_s = df_tmp[[el for el in df_tmp.columns if el>s and el<=steps[i+1]]].sum(axis=1)
        dfs.append(df_s)
    df_tmp = pd.concat(dfs, axis=1)
    df_tmp.columns = [f'>{s}' if s != 0 else '=1' for s in steps[:-1]]
    df_tmp = df_tmp.reset_index()
    df_tmp = df_tmp.melt(id_vars=['donor'], var_name='Size', value_name='Amount clones')
    df_tmp['Donor_A07'] = df_tmp['donor'].apply(lambda x: x if x=='A07' else 'Other') 
    df_tmp = df_tmp.sort_values('Donor_A07', ascending=False)
    plot = sb.stripplot(data=df_tmp, x='Size', y='Amount clones', hue='Donor_A07', palette=cmap_donor_a07)
    plot = sb.boxplot(showmeans=True, meanline=True, meanprops={'color': 'k', 'ls': '-', 'lw': 2},
                      medianprops={'visible': False}, whiskerprops={'visible': False}, 
                      x='Size', y='Amount clones', data=df_tmp, 
                      showfliers=False, showbox=False, showcaps=False)
    sb.despine(ax=plot)
    plot.grid(False)
    plot.set_title(title)
    plot.axhline(1, color='silver', linestyle='--')

In [ ]:
cmap_donor_a07 = {
    'Other': 'firebrick',
    'A07': 'black'
}

def plot_size_amount_clones_split(adata_tmp, title, split_interval):
    df_tmp = adata_tmp.obs[['donor', 'clone_id', 'clone_size_donor']].copy()
    print(len(df_tmp))
    df_tmp = df_tmp.drop_duplicates()
    df_tmp = df_tmp.groupby('donor')['clone_size_donor'].value_counts().unstack().fillna(0)
    steps = [0, 1, 2, 3, 5, 10, 15, 20, 999999]
    dfs = []
    for i, s in enumerate(steps[:-1]):
        df_s = df_tmp[[el for el in df_tmp.columns if el>s and el<=steps[i+1]]].sum(axis=1)
        dfs.append(df_s)
    df_tmp = pd.concat(dfs, axis=1)
    df_tmp.columns = [f'>{s}' if s != 0 else '=1' for s in steps[:-1]]
    df_tmp = df_tmp.reset_index()
    df_tmp = df_tmp.melt(id_vars=['donor'], var_name='Size', value_name='Amount clones')
    df_tmp['Donor_A07'] = df_tmp['donor'].apply(lambda x: x if x=='A07' else 'Other')    
    df_tmp = df_tmp.sort_values('Donor_A07', ascending=False)
    
    fig = plt.figure(figsize=(7, 4))
    gs = gridspec.GridSpec(2, 1, height_ratios=[1, 3], hspace=0.05)
    
    ax_full = fig.add_subplot(gs[:])
    ax_u = fig.add_subplot(gs[0])
    ax_b = fig.add_subplot(gs[1])
    
    plot = sb.stripplot(data=df_tmp[df_tmp['Amount clones']<split_interval[0]],
                        x='Size', y='Amount clones', ax=ax_b,
                        hue='Donor_A07', palette=cmap_donor_a07, )
    plot = sb.boxplot(showmeans=True, meanline=True, meanprops={'color': 'k', 'ls': '-', 'lw': 2},
                      medianprops={'visible': False}, whiskerprops={'visible': False}, 
                      x='Size', y='Amount clones', data=df_tmp, 
                      showfliers=False, showbox=False, showcaps=False, ax=ax_b)
    sb.despine(ax=plot)
    plot.grid(False)
    plot.set_ylabel(None)
    ax_b.axhline(1, color='silver', linestyle='--')
    
    d = 0.03
    ax_u.plot((-d*4/7, +d*4/7), (-d*2, +d*2), transform=ax_u.transAxes, color='silver', clip_on=False, linewidth=0.5)
    ax_b.plot((-d*4/7, +d*4/7), (-d*2/3+1, +d*2/3+1), transform=ax_b.transAxes, color='silver', clip_on=False, linewidth=0.5)
    
    plot = sb.stripplot(data=df_tmp[df_tmp['Amount clones']>split_interval[1]], 
                        x='Size', y='Amount clones',ax=ax_u,
                         hue='Donor_A07', palette=cmap_donor_a07, 
                       order=df_tmp['Size'].unique())
    ax_u.legend().remove()
    ax_b.legend(title='Donor', loc='upper right')
    
    sb.despine(ax=plot, bottom=True)
    plot.grid(False)
    plot.set_xlabel(None)
    plot.set_ylabel(None)
    plot.set_xticklabels([])
    plot.set_title(title)
    
    sb.despine(ax=ax_full, bottom=True, left=True)
    ax_full.set_yticklabels([])
    ax_full.set_xticklabels([])
    ax_full.set_ylabel('Amount clones', labelpad=25)
    ax_full.grid(False)

In [ ]:
adata_covAll = adata[~adata.obs['binding_ct'].isin(['No binding', 'CTELKLSDY', 'FLRGRAYGL', 'RAKFKQLL'])]
plot_size_amount_clones_split(adata_covAll, 'Cov+ - All Donors', split_interval=[100, 190])

plt.tight_layout()
plt.savefig(f'{path_figs}/clones_size_amount_cov+All.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/clones_size_amount_cov+All.png', bbox_inches='tight', dpi=DPI)
plt.show()

In [ ]:
adata_covWoHim = adata[(~adata.obs['binding_ct'].isin(['No binding', 'CTELKLSDY', 'FLRGRAYGL', 'RAKFKQLL']))
                      & (adata.obs['donor']!='HIM')]
plot_size_amount_clones_split(adata_covWoHim, 'Cov+ - without HIM', split_interval=[100, 190])

plt.tight_layout()
plt.savefig(f'{path_figs}/clones_size_amount_cov+woHIM.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/clones_size_amount_cov+woHIM.png', bbox_inches='tight', dpi=DPI)
plt.show()

In [ ]:
adata_ltdAll = adata[adata.obs['binding_ct']=='LTDEMIAQY']
plot_size_amount_clones(adata_ltdAll, 'LTD+ - All Donors')

plt.tight_layout()
plt.savefig(f'{path_figs}/clones_size_amount_ltd+All.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/clones_size_amount_ltd+All.png', bbox_inches='tight', dpi=DPI)
plt.show()

In [ ]:
adata_ltdWoHim = adata[(adata.obs['binding_ct']=='LTDEMIAQY')
                      & (adata.obs['donor']!='HIM')]
plot_size_amount_clones(adata_ltdWoHim, 'LTD+ - without HIM')

plt.tight_layout()
plt.savefig(f'{path_figs}/clones_size_amount_ltd+woHIM.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/clones_size_amount_ltd+woHIM.png', bbox_inches='tight', dpi=DPI)
plt.show()

## Expansion >=

In [ ]:
def plot_line_expansion(df_, label, ax, remove1=False):
    df = df_[df_['clone_id']!='nan'].copy()
    df['clone_id'] = df['clone_id'].astype(str)
    df = df.groupby(['donor','time'])['clone_id'].value_counts(ascending=True).values
    df = df[df!=0]
    if remove1:
        df = df[df!=1]
    
    df = sorted(df)    
    
    xs = list(set(list(df)))
    xs.append(max(xs)+1)
    
    ys = []
    for x in xs:
        y = np.sum(df>=x) / len(df)
        ys.append(y)
    df_xy = pd.DataFrame({'x': xs, 'y': ys})
    
    sb.lineplot(data=df_xy, x='x', y='y', ax=ax, label=label)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
plot_line_expansion(adata.obs, 'All cells', ax)
plot_line_expansion(adata[adata.obs['leiden_CD8']=='5'].obs, 'Naive cells', ax)
plot_line_expansion(adata[adata.obs['leiden_CD8'].isin(['5', '7', '9', '12', '11'])
                         ].obs, 'All wo 5, 7, 9, 12, 11', ax)

ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('Cell count per clone')
ax.set_ylabel('Proportion of clones >= size')
sb.despine(ax=ax)
ax.grid(False)

plt.tight_layout()
plt.savefig(f'{path_figs}/expansion_proportion.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/expansion_proportion.png', bbox_inches='tight', dpi=DPI)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
plot_line_expansion(adata.obs, 'All cells', ax)
plot_line_expansion(adata[adata.obs['binding_ct']!='No binding'].obs, 'Dex+', ax)
plot_line_expansion(adata[adata.obs['binding_ct']=='LTDEMIAQY'].obs, 'LTD+', ax)

ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('Cell count per clone')
ax.set_ylabel('Proportion of clones >= size')
sb.despine(ax=ax)
ax.grid(False)

plt.tight_layout()
plt.savefig(f'{path_figs}/expansion_proportion_binders.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/expansion_proportion_binders.png', bbox_inches='tight', dpi=DPI)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
plot_line_expansion(adata.obs, 'All cells', ax, remove1=True)
plot_line_expansion(adata[adata.obs['leiden_CD8']=='5'].obs, 'Naive cells', ax, remove1=True)
plot_line_expansion(adata[adata.obs['leiden_CD8'].isin(['5', '7', '9', '12', '11'])
                         ].obs, 'All wo 5, 7, 9, 12, 11', ax, remove1=True)

ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('Cell count per clone')
ax.set_ylabel('Proportion of clones >= size wo clone size 1')
sb.despine(ax=ax)
ax.grid(False)

plt.tight_layout()
plt.savefig(f'{path_figs}/expansion_proportion_wo1.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/expansion_proportion_wo1.png', bbox_inches='tight', dpi=DPI)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
plot_line_expansion(adata.obs, 'All cells', ax, remove1=True)
plot_line_expansion(adata[adata.obs['binding_ct']!='No binding'].obs, 'Dex+', ax, remove1=True)
plot_line_expansion(adata[adata.obs['binding_ct']=='LTDEMIAQY'].obs, 'LTD+', ax, remove1=True)

ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('Cell count per clone')
ax.set_ylabel('Proportion of clones >= size wo clone size 1')
sb.despine(ax=ax)
ax.grid(False)

plt.tight_layout()
plt.savefig(f'{path_figs}/expansion_proportion_binders_wo1.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/expansion_proportion_binders_wo1.png', bbox_inches='tight', dpi=DPI)
plt.show()

## DEG Plots

In [ ]:
# we lost ccr7 due to naming conflicts => re-add it to adata
adata_early = sc.read(f'../../data/dextramer/01_dex_merged.h5ad')[:, ['CCR7']]
adatas = []
for i in range(1, 4):
    path_gex = f'../../data/20231017/GEX/gex_mixed_run_{i}_feature_bc_matrix.h5'
    adata_tmp = sc.read_10x_h5(path_gex, gex_only=False)
    adata_tmp.var_names_make_unique()
    adata_tmp.obs['CCR7'] = adata_tmp[:, 'CCR7-1'].X.A
    filter_ids = cite_ids.tolist() + adata.uns['epitopes'].tolist() + ['CCR7-1', 'CD62L', 'CXCR3', 'CD45RA']
    adata_tmp = adata_tmp[:, [el for el in adata_tmp.var_names if el not in filter_ids]]
    sc.pp.log1p(adata_tmp)
    adatas.append(adata_tmp)
adata_mixed = adatas[0].concatenate(adatas[1:])[:, ['CCR7']]
adata_mixed = adata_early.concatenate(adata_mixed)
adata_mixed = adata_mixed[adata.obs.index].copy()

adata = sc.read('../../data/dextramer/02_dex_annotated_cd8.h5ad')
adata.uns['log1p']['base'] = None
adata_tmp = ann.AnnData(X=sparse.csr_matrix(np.concatenate([adata.X.A, adata_mixed.X.A], axis=1)),
                          obs=adata.obs, uns=adata.uns, obsm=adata.obsm)
adata_tmp.var_names = adata.var_names.tolist() + ['CCR7']
adata = adata_tmp.copy()
adata.obs.pop('CCR7')
adata.obs['CCR7_c'] = adata_mixed.obs['CCR7']
adata.obs.loc[adata.obs['CXCR3'].isna(), 'CCR7_c'] = np.nan

In [ ]:
def double_histogram_dotplot(adata, top_genes, groupby, save_name=None, cluster_order=None):
    # This Code is highjacking the parts of functions from scanpy, scipy, and matplotlib. 
    # If you re-use it, that's between you and your deity.     
    import scipy.cluster.hierarchy as sch
    from scipy.spatial import distance
    
    # Mean expression per cluster for gene dendrogram
    df_genes = pd.DataFrame(data=adata[:, top_genes].X.A, columns=top_genes, index=adata.obs.index)
    df_genes[groupby] = adata.obs[groupby]
    df_counts = df_genes.groupby(groupby)[top_genes].mean()
    
    # Dendrogram without plotting
    correlation_matrix = df_counts.corr(method='pearson')
    correlation_condensed = distance.squareform(1 - correlation_matrix)
    z_var = sch.linkage(correlation_condensed, method='complete')
    dendro_info = sch.dendrogram(z_var, labels=list(top_genes), no_plot=True, 
                                 color_threshold=0, above_threshold_color='k')
    
    # Normal Dotplot with dummy brackets on top for extra axis
    plot = sc.pl.rank_genes_groups_dotplot(adata, var_names=dendro_info['ivl'], dendrogram=cluster_order is None,
                                           categories_order=cluster_order,
                                           show=False, 
                                       var_group_labels=[''], var_group_positions=[(4,10)], 
                                          key=f'rank_genes_groups_CD8'
                                          )
    
    # Delete dummy Bracket
    plot['gene_group_ax'].cla()
    plot['gene_group_ax'].grid(False)
    sb.despine(ax=plot['gene_group_ax'], bottom=True, left=True)
    
    # Add custom dendrogram: Scale the x-Coordinates, put it within the Group Genes axis, and rescale this axis
    scale_factor = (len(top_genes)-0.5)/(np.array(dendro_info['icoord']).max())
    sch._plot_dendrogram(icoords=np.array(dendro_info['icoord'])*scale_factor, dcoords=np.array(dendro_info['dcoord'])+0.03, 
                        ivl=dendro_info['ivl'],
                        p=30, n=len(top_genes), mh=max(z_var[:, 2]),
                       orientation='top', no_labels=True, color_list=dendro_info['color_list'], ax=plot['gene_group_ax'])
    plot['gene_group_ax'].set_xlim([1, len(top_genes)+1])
    pos = plot['gene_group_ax'].get_position()
    plot['gene_group_ax'].set_position([pos.x0, pos.y0, pos.x1-pos.x0, 0.5])

    # Re-adjust the labels since they were lost while plotting
    plot['mainplot_ax'].set_xticks([el+0.5 for el in range(0, len(top_genes))])
    plot['mainplot_ax'].set_xlim([0, len(top_genes)])
    _ = plot['mainplot_ax'].set_xticklabels(dendro_info['ivl'])
    
    if save_name is not None:
        plt.savefig(f'{path_figs}/{save_name}.pdf', bbox_inches='tight', dpi=dpi)
        plt.savefig(f'{path_figs}/{save_name}.png', bbox_inches='tight', dpi=dpi)
    plt.show()

In [ ]:
markers_old = list(set([el for l in adata.uns['rank_genes_groups_CD8']['names'][:5].tolist() for el in l]))
markers_exclude = ['ALOX5AP', 'RPS8', 'RPL13', 'RPL8', 'RPS13', 'RPLP1', 'RPL32', 'RPS4Y1',
                   'CD2', 'GAPDH', 'PPP1R14B', 'IFITM1', 'CMC1', 'CST7', 'ZFP36L2', 'SRSF3',
                   'CXCR4', 'HLA-C', 'MT-ND3', 'MT-CO1', 'MALAT1']
markers_include = ['SELL', 'TCF7', 'MKI67', 'UBE2L6', 'PRF1', 'LTB', 'KLRD1', 'FGFBP2', 'CCR7']

markers_updated = [el for el in markers_old + markers_include if el not in markers_exclude]
    
double_histogram_dotplot(adata, markers_updated, groupby=f'leiden_CD8', save_name='gex_gene_dendrogram')

In [ ]:
costum_order = ['11', '12', '6', '8', '10', '0', '1', '9', '3', '5', '7', '2', '4']

markers_cite = [#'CD62L', 'CD45RA', 'CCR7',
                'Hu.CD45RA', 'Hu.CD183', 'Hu.CD57', 'Hu.GPR56', 'Hu.CD62L',
                'Hu.HLA.DR', 'Hu.CD38_HIT2', 'Hu.CD103', 'Hu.CD161', 'Hu.CD26', 'Hu.TCR.Va7.2']
adata_cite = ann.AnnData(obs=adata.obs[['leiden_CD8']], X=adata.obs[[f'clr_{el}' for el in adata.uns['cite_ids']]].values)
adata_cite.var_names = adata.uns['cite_ids']
adata_cite.X = sparse.csr_matrix(adata_cite.X)
adata_cite.uns['rank_genes_groups_CD8'] = adata.uns['rank_genes_groups_leiden_cite'].copy()
double_histogram_dotplot(adata_cite[adata.obs[cite_ids[0]].notna()], 
                         markers_cite, groupby=f'leiden_CD8', save_name='cite_gene_dendrogram',
                        cluster_order=costum_order)

In [ ]:
def clr(x):
    x = x/np.exp(np.log1p(x).sum() / x.shape[0])
    x = np.log1p(x)
    return x

costum_order = ['11', '6', '8', '10', '0', '1', '9', '3', '5', '7', '2', '4']

c = 'CCR7_c'
adata.obs.loc[~adata.obs[c].isna(), f'clr_{c}'] = clr(adata[~adata.obs[c].isna()].obs[c].values)
markers_cite = ['CD62L', 'CD45RA', 'CCR7_c']
custom_cite_ids = ['CCR7_c', 'CD62L', 'CXCR3', 'CD45RA']
adata_cite = ann.AnnData(obs=adata.obs[['leiden_CD8', 'clr_CCR7_c']], X=adata.obs[[f'clr_{el}' for el in custom_cite_ids]].values)
adata_cite.var_names = custom_cite_ids
adata_cite.X = sparse.csr_matrix(adata_cite.X)
adata_cite.uns['rank_genes_groups_CD8'] = adata.uns['rank_genes_groups_leiden_cite'].copy()
adata_cite = adata_cite[adata_cite.obs['clr_CCR7_c'].notna()]
sc.pl.rank_genes_groups_dotplot(adata_cite, 
                                categories_order=costum_order,
                                var_names=markers_cite, dendrogram=False,
                                key=f'rank_genes_groups_CD8', show=False)
plt.tight_layout()
plt.savefig(f'{path_figs}/cite_custom_gene_dendrogram.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/cite_custom_gene_dendrogram.png', bbox_inches='tight', dpi=DPI)
plt.show()

### Clone Heatmaps

In [ ]:
epi_order = ['LTDEMIAQY', 'YLQPRTFLL', 'RLQSLQTYV', 'KCYGVSPTK', 'QYIKWPWYI', 'NYNYLYRLF',
            'QPYRVVVL', 'YTNSFTRGVY', 
            'ATDSLNNEY','CTELKLSDY', 'FLRGRAYGL', 'RAKFKQLL']
cmap_colors = 'seismic'
cmap_colors = 'RdBu_r'
cmap_colors = 'RdYlBu_r'

def umi_plot(adata_tmp, epitopes_show, y=3.45, x=3.55):
    df_tmp = adata_tmp.obs[['clone_id', 'binding_ct'] + epitopes.tolist()]
    df_tmp = df_tmp[(df_tmp['binding_ct']!='No binding') & (df_tmp['clone_id']!='nan')]
    df_tmp['binding_ct'] = df_tmp['binding_ct'].astype(str)
    df_tmp['clone_id'] = df_tmp['clone_id'].astype(str)
    df_tmp = df_tmp.groupby(['clone_id', 'binding_ct']).mean()
    df_tmp = df_tmp.reset_index().set_index('clone_id')
    tmps = []
    if len([el for el in df_tmp['binding_ct'].unique() if el not in epi_order]) != 0:
        print([el for el in df_tmp['binding_ct'].unique() if el not in epi_order])
        raise ValueError('Epitope missing in order')
    for e in [el for el in epi_order if el in df_tmp['binding_ct'].unique()]:
        tmp = df_tmp[df_tmp['binding_ct']==e]
        tmp = tmp.sort_values([e, 'clone_id'], ascending=[False, True])
        tmps.append(tmp)
    df_tmp = pd.concat(tmps)
    df_counts = df_tmp[[el for el in df_tmp.columns if el != 'binding_ct']].copy()
    df_counts = df_counts[epitopes_show].transpose()
    df_counts = np.log10(df_counts+1)

    size_x = len(df_tmp)/x
    size_y = len(df_counts)/y
    set_x = 0.2
    set_y = 1/3.6
    fig = plt.figure(figsize=(size_x+set_x, size_y+set_y), dpi=DPI)

    gs = mpl.gridspec.GridSpec(figure=fig, nrows=2, ncols=2, 
                               width_ratios=[set_x, size_x],
                               height_ratios=[set_y, size_y],
                               hspace=0.025, wspace=0.025)
    ax_anno = fig.add_subplot(gs[1])
    ax_cbar = fig.add_subplot(gs[2])
    ax_umi = fig.add_subplot(gs[3])

    sb.heatmap(data=df_counts, ax=ax_umi, cmap=cmap_colors, cbar=False, linewidths=0.01, linecolor='dimgrey')
    ax_umi.yaxis.tick_right()

    ax_umi.set_yticklabels(ax_umi.get_yticklabels(), rotation=0)
    ax_umi.set_xticklabels([])
    ax_umi.tick_params(axis='both', which='both', length=0, pad=7)
    ax_umi.set_xlabel(None)

    df_spec = df_tmp[['binding_ct']].copy()
    df_spec['binding_ct'] = df_spec['binding_ct'].apply(lambda x: list(palette_epis.keys()).index(x))
    df_spec = df_spec.transpose()

    cmap = clrs.LinearSegmentedColormap.from_list('dextramer', 
                                                  [(k/(len(palette_epis_int)-1), v) for k, v in palette_epis_int.items()])
    sb.heatmap(data=df_spec, ax=ax_anno, cbar=False, cmap=cmap, vmin=0, vmax=len(palette_epis_int)-1)
    ax_anno.set_xlabel(None)
    ax_anno.xaxis.tick_top()
    ax_anno.set_xticklabels(ax_anno.get_xticklabels(), rotation=90)
    ax_anno.set_yticklabels([])
    ax_anno.tick_params(axis='both', which='both', length=0, pad=7)


    norm = clrs.Normalize(vmin=df_counts.min().min(), vmax=df_counts.max().max())
    cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap_colors), cax=ax_cbar, orientation='vertical')
    ax_cbar.set_ylabel('log$_{10}$(UMI+1)')
    #ax_cbar.set_ylabel('UMI')
    ax_cbar.yaxis.set_ticks_position('left')
    ax_cbar.yaxis.set_label_position('left')
    return df_counts

In [ ]:
adata_tmp = adata[(adata.obs['donor']=='A15')
                 & (adata.obs['experiment']=='first_experiment')]
epitopes_show = ['LTDEMIAQY', 'YTNSFTRGVY', 'YLQPRTFLL', 'RLQSLQTYV', 'VLNDILSRL',
       'KIADYNYKL', 'KCYGVSPTK', 'QYIKWPWYI', 'NYNYLYRLF', 'SPRRARSVA', 'FPQSAPHGV', 'QPYRVVVL',
       'IYKTPPIKDF']
umi_plot(adata_tmp, epitopes_show)
plt.savefig(f'{path_figs}/umi_vs_clone_a15_first_experiment_{cmap_colors}.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/umi_vs_clone_a15_first_experiment_{cmap_colors}.png', bbox_inches='tight', dpi=DPI)
plt.show()

In [ ]:
adata_tmp = adata[(adata.obs['donor']=='A15')
                 & (adata.obs['experiment']=='third_experiment')]
epitopes_show = ['LTDEMIAQY', 'YTNSFTRGVY', 'NYNYLYRLF', 'QPYRVVVL', 
                 'TFEYVSQPFLMDLE', 'ATDSLNNEY','CTELKLSDY', 'FLRGRAYGL', 'RAKFKQLL']
umi_plot(adata_tmp, epitopes_show)
plt.savefig(f'{path_figs}/umi_vs_clone_a15_third_experiment.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/umi_vs_clone_a15_third_experiment.png', bbox_inches='tight', dpi=DPI)
plt.show()

In [ ]:
adata_tmp = adata[(adata.obs['donor']=='A29')
                 & (adata.obs['experiment']=='first_experiment')]
epitopes_show = ['LTDEMIAQY', 'YTNSFTRGVY', 'YLQPRTFLL', 'RLQSLQTYV', 'VLNDILSRL',
       'KIADYNYKL', 'KCYGVSPTK', 'QYIKWPWYI', 'NYNYLYRLF', 'SPRRARSVA', 'FPQSAPHGV', 'QPYRVVVL',
       'IYKTPPIKDF']
umi_plot(adata_tmp, epitopes_show, x=3.5)
plt.savefig(f'{path_figs}/umi_vs_clone_a29_first_experiment_{cmap_colors}.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/umi_vs_clone_a29_first_experiment_{cmap_colors}.png', bbox_inches='tight', dpi=DPI)
plt.show()

In [ ]:
adata_tmp = adata[(adata.obs['donor']=='A07')
                 & (adata.obs['time']=='extra')
                 #& (adata.obs['binding_ct'].isin(['ATDSLNNEY','CTELKLSDY', 'FLRGRAYGL', 'RAKFKQLL']))
                 ]
epitopes_show = ['ATDSLNNEY','CTELKLSDY', 'FLRGRAYGL', 'RAKFKQLL']
umi_plot(adata_tmp, epitopes_show, y=3.05)
plt.savefig(f'{path_figs}/umi_vs_clone_a07_extra_{cmap_colors}.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/umi_vs_clone_a07_extra_{cmap_colors}.png', bbox_inches='tight', dpi=DPI)
plt.show()

In [ ]:
adata_tmp = adata[(adata.obs['donor']=='A15')
                 & (adata.obs['time']=='extra')
                 #& (adata.obs['binding_ct'].isin(['ATDSLNNEY','CTELKLSDY', 'FLRGRAYGL', 'RAKFKQLL']))
                 ]
epitopes_show = ['ATDSLNNEY','CTELKLSDY', 'FLRGRAYGL', 'RAKFKQLL']
umi_plot(adata_tmp, epitopes_show, y=3.05)
plt.savefig(f'{path_figs}/umi_vs_clone_a15_extra.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/umi_vs_clone_a15_extra.png', bbox_inches='tight', dpi=DPI)
plt.show()

In [ ]:
adata_tmp = adata[(adata.obs['donor']=='HIM')
                 & (adata.obs['time']=='X3')
                 & (adata.obs['binding_ct'].isin(['ATDSLNNEY','CTELKLSDY', 'FLRGRAYGL', 'RAKFKQLL']))
                & (adata.obs['ATDSLNNEY'].notna())
                 ]
epitopes_show = ['ATDSLNNEY','CTELKLSDY', 'FLRGRAYGL', 'RAKFKQLL']
umi_plot(adata_tmp, epitopes_show, y=3.05, x=2.6)
plt.savefig(f'{path_figs}/umi_vs_clone_him_extra.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/umi_vs_clone_him_extra.png', bbox_inches='tight', dpi=DPI)
plt.show()

### Specificity by Leiden Cluster

In [ ]:
len(specs)

In [ ]:
n_rows = 5
n_cols = 3


df_tmp = adata.obs.groupby(f'leiden_{cell}')['binding_ct'].value_counts(normalize=True)
df_tmp = pd.DataFrame(df_tmp)
df_tmp = df_tmp.rename(columns={binding_mode: 'count'})
df_tmp = df_tmp.reset_index()

fig, axes = plt.subplots(ncols=n_cols, nrows=n_rows, figsize=(n_cols * 4, n_rows * 3))
axes = axes.reshape(-1)

for i, e in enumerate(specs):
    df_epi = df_tmp[df_tmp[binding_mode]==e]
    sb.barplot(data=df_epi, y='count', x=f'leiden_{cell}', ax=axes[i],
               palette=palette_leiden,
               order=leiden_dpt_order)
    sb.despine(ax=axes[i])
    axes[i].set_ylabel("Fraction within Leiden")
    axes[i].set_title(e)
    axes[i].grid(False)

    #axes[i].legend().remove()
axes[-1].axis('off')
fig.tight_layout()
plt.savefig(f'{path_figs}/fraction_specificity_over_leiden.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/fraction_specificity_over_leiden.png', bbox_inches='tight', dpi=DPI)
plt.show()

In [ ]:
n_rows = 5
n_cols = 3


df_tmp = adata.obs.groupby(f'leiden_{cell}')['binding_ct'].value_counts(normalize=False)
df_tmp = pd.DataFrame(df_tmp)
df_tmp = df_tmp.rename(columns={binding_mode: 'count'})
df_tmp = df_tmp.reset_index()

n_binding_cells = len(adata[adata.obs['binding_ct']!='No binding'])
df_tmp['count'] = df_tmp['count'] / n_binding_cells

fig, axes = plt.subplots(ncols=n_cols, nrows=n_rows, figsize=(n_cols * 4, n_rows * 3))
axes = axes.reshape(-1)

for i, e in enumerate(specs):
    df_epi = df_tmp[df_tmp[binding_mode]==e]
    sb.barplot(data=df_epi, y='count', x=f'leiden_{cell}', ax=axes[i],
               palette=palette_leiden,
               order=leiden_dpt_order)
    sb.despine(ax=axes[i])
    axes[i].set_ylabel("Fraction of Dex+")
    axes[i].set_title(e)
    axes[i].grid(False)

    #axes[i].legend().remove()
axes[-1].axis('off')
fig.tight_layout()
plt.savefig(f'{path_figs}/fraction_to_dex+_specificity_over_leiden.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/fraction_to_dex+_specificity_over_leiden.png', bbox_inches='tight', dpi=DPI)
plt.show()

In [ ]:
n_rows = 5
n_cols = 3

df_tmp = adata.obs.groupby(f'leiden_{cell}')['binding_ct'].value_counts(normalize=False)
df_tmp = pd.DataFrame(df_tmp)
df_tmp = df_tmp.rename(columns={binding_mode: 'count'})
df_tmp = df_tmp.reset_index()

fig, axes = plt.subplots(ncols=n_cols, nrows=n_rows, figsize=(n_cols * 4, n_rows * 3))
axes = axes.reshape(-1)

for i, e in enumerate(specs):
    df_epi = df_tmp[df_tmp[binding_mode]==e].copy()
    df_epi['count'] = df_epi['count'] / df_epi['count'].sum()
    sb.barplot(data=df_epi, y='count', x=f'leiden_{cell}', ax=axes[i],
               palette=palette_leiden,
               order=leiden_dpt_order)
    sb.despine(ax=axes[i])
    axes[i].set_ylabel("Distribution over Cluster")
    axes[i].set_title(e)
    axes[i].grid(False)

    #axes[i].legend().remove()
axes[-1].axis('off')
fig.tight_layout()
plt.savefig(f'{path_figs}/fraction_specificity_over_leiden2.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/fraction_specificity_over_leiden2.png', bbox_inches='tight', dpi=DPI)
plt.show()

## IFN vs cellcount LTD

In [ ]:
rcParams['figure.figsize'] = (5, 5)
for t1, t2 in [['T1', 'T1'], ['S1', 'S1'], ['T1', 'S1'], [None, None]]:
    adata_tmp = adata[(adata.obs['binding_ct']=='LTDEMIAQY')]
    if t1 is not None:
        adata_tmp = adata_tmp[adata_tmp.obs['time']==t1]
    cell_counts = pd.DataFrame(adata_tmp.obs['clone_id'].value_counts())
    cell_counts.columns = ['# Cells']
    
    adata_tmp = adata[(adata.obs['binding_ct']=='LTDEMIAQY')]
    if t2 is not None:
        adata_tmp = adata_tmp[adata_tmp.obs['time']==t2]
    inf_score = pd.DataFrame(adata_tmp.obs.groupby('clone_id')['ifn_seumois'].mean())
    
    df_tmp = pd.concat([cell_counts, inf_score], axis=1)
    df_tmp = df_tmp[df_tmp['ifn_seumois'].notna() & df_tmp['# Cells'].notna()]
    
    plot = sb.scatterplot(data=df_tmp, x='ifn_seumois', y='# Cells', color='firebrick')
    plot = sb.regplot(data=df_tmp, x='ifn_seumois', y='# Cells', ax=plot, color='grey', scatter=False, 
                      line_kws={'zorder': -1})
    sb.despine(ax=plot)
    plot.grid(None)
    
    corr, pval = stats.pearsonr(df_tmp['ifn_seumois'], df_tmp['# Cells'])
    if t1 is None:
        title = 'pooled'
    else:
        title = f'# Cells: {t1} - Inf Seumois: {t2}'
    title += f'\nCorr: {corr:.3f}   -   P-val: {pval:.3f}'
    plot.set_title(title)

    plt.tight_layout()
    plt.savefig(f'{path_figs}/infSeumois{t2}_vs_#Cells{t1}.pdf', bbox_inches='tight', dpi=DPI)
    plt.savefig(f'{path_figs}/infSeumois{t2}_vs_#Cells{t1}.png', bbox_inches='tight', dpi=DPI)
    plt.show()

## CT distribution

In [ ]:
import skbio

fig, ax = plt.subplots(figsize=(6, 6))
df_tmp = adata[(adata.obs[binding_mode]=='LTDEMIAQY')
              & (adata.obs['clone_id']!='nan')
              & (adata.obs['time'].isin(['T3', 'X3']))].obs[['donor', 'clone_id']]
df_tmp = df_tmp.groupby('donor')['clone_id'].value_counts().unstack().fillna(0)

df_tmp.columns = [el for el in df_tmp.columns]
for col in df_tmp:
    clone_counts = df_tmp[col]
    if np.sum(clone_counts>0)>1:
        for donor in clone_counts[clone_counts>0].index:
            df_tmp[f'{col}_{donor}'] = [0.0] * len(clone_counts)
            df_tmp.loc[donor, f'{col}_{donor}'] = clone_counts.loc[donor]
        df_tmp = df_tmp.drop(columns=[col])

# Ordering the df
stack_order = df_tmp.max(axis=0).sort_values(ascending=False).index
df_tmp = df_tmp[stack_order]
donor_order = df_tmp.sum(axis=1).sort_values().index
df_tmp = df_tmp.loc[donor_order]

clone_size = np.sum(df_tmp).values
color = sb.color_palette("Blues", as_cmap=True)(clone_size/max(clone_size))

df_tmp.plot(kind='bar', stacked=True, color=color, 
            ax=ax)

n_cts_by_donor = (df_tmp>0).sum(axis=1)
for i, (donor, n) in enumerate(n_cts_by_donor.items()):
    gini = skbio.diversity.alpha.gini_index(df_tmp.loc[donor][df_tmp.loc[donor]>0].values, method='trapezoids')
    ax.text((i+0.5)/len(n_cts_by_donor), 1, f'n={n}\n{gini:.2f}', ha='center', 
            transform=ax.transAxes)


ax.legend().remove()
sb.despine(ax=ax)
ax.grid(False)
ax.set_ylabel('Absolute cell numbers')
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
ax.set_xlabel(None)
plt.tight_layout()
plt.savefig(f'{path_figs}/stacked_clonotypes_by_donor_X3T3.pdf', bbox_inches='tight', dpi=DPI)
plt.savefig(f'{path_figs}/stacked_clonotypes_by_donor_X3T3.png', bbox_inches='tight', dpi=DPI)
plt.show()

# Numbers

In [ ]:
print('Total amount of cells')
len(adata)

In [ ]:
print('Total clones by specificity in A04-08-15-07-A16')
adata[adata.obs['donor'].isin(['A04', 'A08', 'A15', 'A07', 'A16'])].obs.groupby('binding_ct')['clone_id'].nunique()

In [ ]:
adata[adata.obs['binding_ct']=='LTDEMIAQY'].obs.groupby('donor')['clone_id'].nunique()

In [ ]:
adata[(adata.obs['binding_ct']=='LTDEMIAQY') 
     & adata.obs['donor'].isin(['A04', 'A08', 'A15', 'A07', 'A16'])].obs.groupby('clone_id')['donor'].nunique().sort_values()